## Contributors
### Jay Upadhyay
### 
###
###
###

# Libraries

In [96]:
from tensorflow import keras
from keras import utils
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import numpy as np
import h5py
import pandas as pd
import seaborn as sns

In [97]:
df_orig=pd.read_csv("adult.csv")
df_orig.dropna(inplace=True)
df_orig

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


# 1. Project Proposal


We pulled the data from https://archive-beta.ics.uci.edu/dataset/2/adult .

# 2. Data Collection and data cleaning


## Changing Education to Numeric

In [98]:
df_orig.education.unique()


array([' Bachelors', ' HS-grad', ' 11th', ' Masters', ' 9th',
       ' Some-college', ' Assoc-acdm', ' Assoc-voc', ' 7th-8th',
       ' Doctorate', ' Prof-school', ' 5th-6th', ' 10th', ' 1st-4th',
       ' Preschool', ' 12th'], dtype=object)

#### Note. I want to come back to this and bin it into appropriate 
I also chose to exclude this for now because I believe education-num already represents it numerically.

In [99]:
# df_orig['education'].replace([ ' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',' 12th',' HS-grad',
#                               ' Assoc-voc',' Assoc-acdm','Some-college',' Bachelors', ' Masters',
#                               ' Prof-school' ,' Doctorate'],
#                         [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], inplace=True)


In [100]:
df_orig

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## Changing marital status to numeric

In [101]:
df_orig["marital-status"].unique()

array([' Never-married', ' Married-civ-spouse', ' Divorced',
       ' Married-spouse-absent', ' Separated', ' Married-AF-spouse',
       ' Widowed'], dtype=object)

In [102]:
df_orig["marital-status"].replace([' Never-married', ' Married-civ-spouse', ' Divorced',
       ' Married-spouse-absent', ' Separated', ' Married-AF-spouse',
       ' Widowed'],range(1,8), inplace=True)
df_orig



,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,1,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,2,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,3,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,2,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,2,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,2,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,2,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,7,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,1,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## Occupation

I don't like the ? mark section.

In [103]:
df_orig["occupation"].unique()

array([' Adm-clerical', ' Exec-managerial', ' Handlers-cleaners',
       ' Prof-specialty', ' Other-service', ' Sales', ' Craft-repair',
       ' Transport-moving', ' Farming-fishing', ' Machine-op-inspct',
       ' Tech-support', ' ?', ' Protective-serv', ' Armed-Forces',
       ' Priv-house-serv'], dtype=object)

In [104]:
df_orig["occupation"].replace([' Adm-clerical', ' Exec-managerial', ' Handlers-cleaners',
       ' Prof-specialty', ' Other-service', ' Sales', ' Craft-repair',
       ' Transport-moving', ' Farming-fishing', ' Machine-op-inspct',
       ' Tech-support', ' ?', ' Protective-serv', ' Armed-Forces',
       ' Priv-house-serv'],range(0,15), inplace=True)
df_orig

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,1,0,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,2,1,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,3,2,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,2,2,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,2,3,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,2,10,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,2,9,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,7,0,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,1,0,Own-child,White,Male,0,0,20,United-States,<=50K


## Relationship conversion to numeric

I don't like other relative section here.


In [105]:
df_orig["relationship"].unique()

array([' Not-in-family', ' Husband', ' Wife', ' Own-child', ' Unmarried',
       ' Other-relative'], dtype=object)

In [106]:
df_orig["relationship"].replace([' Not-in-family', ' Husband', ' Wife', ' Own-child', ' Unmarried',
       ' Other-relative'],range(0,6), inplace=True)
df_orig

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,1,0,0,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,2,1,1,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,3,2,0,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,2,2,1,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,2,3,2,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,2,10,2,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,2,9,1,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,7,0,4,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,1,0,3,White,Male,0,0,20,United-States,<=50K


## Race conversion to numeric


In [107]:
df_orig["race"].unique()

array([' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo',
       ' Other'], dtype=object)

In [108]:
df_orig["race"].replace([' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo',
       ' Other'],range(0,5), inplace=True)
df_orig

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,1,0,0,0,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,2,1,1,0,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,3,2,0,0,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,2,2,1,1,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,2,3,2,1,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,2,10,2,0,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,2,9,1,0,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,7,0,4,0,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,1,0,3,0,Male,0,0,20,United-States,<=50K


## Sex conversion to numeric

In [109]:
df_orig["sex"].unique()

array([' Male', ' Female'], dtype=object)

In [110]:
df_orig["sex"].replace([' Male', ' Female'],range(0,2), inplace=True)
df_orig

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,1,0,0,0,0,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,2,1,1,0,0,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,3,2,0,0,0,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,2,2,1,1,0,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,2,3,2,1,1,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,2,10,2,0,1,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,2,9,1,0,0,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,7,0,4,0,1,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,1,0,3,0,0,0,0,20,United-States,<=50K


## Correlation Matrix

In [111]:
# df_orig.iloc[:,[3,8,9,13,14]]
# df_small = df_orig.iloc[:,[8,9,13,14]]
# correlation_mat = df_small.corr()
# sns.heatmap(correlation_mat, annot = True)
# plt.show()

# 3. EDA

# 4. Techniques Used

# 5. Project Proposal